In [1]:
import requests
import pandas as pd

# Ampliando df para visualizar todas as colunas
pd.set_option('display.max_columns', None)

In [2]:
import requests

# Definindo a URL da API
url = "https://api.elsevier.com/content/search/scopus"

# Parâmetros para a requisição
params = {
    "query": 'TITLE-ABS-KEY ( "marketing" AND ( "data-driven" OR "data driven" ) )',
    "apiKey": "324deadcc7d805c701b628ac20f685c2",
    "count": 200,  # Definindo o número máximo de registros por página
    "start": 0  # Começando a partir do primeiro registro
}

all_results = []  # Lista para armazenar todos os resultados

while True:
    # Fazendo a requisição GET
    response = requests.get(url=url, params=params)

    # Verificando se a requisição foi bem sucedida
    if response.status_code == 200:
        # Transformando a resposta em um objeto JSON
        data = response.json()

        try:
            # Adicionando os resultados desta página à lista geral
            all_results.extend(data['search-results']['entry'])
        except KeyError:
            # Se a chave 'entry' não estiver presente na resposta, apenas continue para a próxima iteração
            pass

        # Verificando se há mais resultados para recuperar
        if int(data['search-results']['opensearch:itemsPerPage']) == 0:
            # Se não houver mais resultados, interrompa o loop
            break

        # Atualizando o índice do próximo registro a ser recuperado
        params['start'] += params['count']
    else:
        # print("Erro na requisição", response.status_code)
        break

# Imprimindo o número total de resultados obtidos
# print("Total de resultados:", len(all_results))


In [42]:
# Removendo colunas desnecessárias
columns = ['@_fa', 
           'link',
           'prism:url',
           'eid',
           'pii',
           'subtype',
           'source-id',
           'openaccess',
           'openaccessFlag',
           'article-number',
           'freetoread',
           'freetoreadLabel',
           'prism:aggregationType',
           'prism:publicationName',
           'prism:issn',
           'prism:volume',
           'prism:issueIdentifier',
           'prism:pageRange',
           'prism:coverDisplayDate',
           'prism:isbn',
           'prism:eIssn',
           'pubmed-id']


df_results = df_results.drop(columns=columns)



In [44]:
# Tratando a coluna dc:identifier - Removendo o prefixo 'SCOPUS_ID:'
df_results['dc:identifier'] = df_results['dc:identifier'].str.replace('SCOPUS_ID:', '')

# Tratando a coluna affiliation - Transformando em uma lista de strings
# Removendo [] da coluna affiliation
df_results['affiliation'] = df_results['affiliation'].apply(lambda x: [aff['affilname'] for aff in x][0] if isinstance(x, list) and x else None)

# Filtrando para registros que são artigos
df_results = df_results[df_results['subtypeDescription'] == 'Article']

,dc:identifier,dc:title,dc:creator,prism:coverDate,prism:doi,citedby-count,affiliation,subtypeDescription
0,85191180283,Developing customer analytics capability in fi...,Mehrabi H.,2024-05-01,10.1016/j.indmarman.2024.04.009,0,Kingston Business School,Article
1,85190999153,Unlocking the potential of Industrial Internet...,Endres H.,2024-05-01,10.1016/j.indmarman.2024.03.006,0,Universität Regensburg,Article
2,85189010126,Future applications of generative large langua...,Filippo C.,2024-05-01,10.1016/j.technovation.2024.103002,1,Università di Pisa,Article
3,85188689332,Mathematical analysis of big data analytics un...,Jan N.,2024-05-01,10.1016/j.asoc.2024.111481,1,Korea National University of Transportation,Article
4,85182892371,GNN-IR: Examining graph neural networks for in...,Park J.,2024-05-01,10.1016/j.jretconser.2024.103705,1,University of Toronto,Article


In [46]:
# Adicionando a coluna 'abstract' ao DataFrame

# Função para obter o resumo com base no DOI
def get_abstract(doi):
    url = f"https://api.elsevier.com/content/abstract/doi/{doi}?apiKey=324deadcc7d805c701b628ac20f685c2"
    headers = {'Accept': 'application/json'}
    response = requests.get(url, headers=headers)
    data = response.json()
    try:
        abstract_content = data["abstracts-retrieval-response"]["item"]["bibrecord"]["head"]["abstracts"]
        return abstract_content
    except KeyError:
        return None

# Itera sobre os valores da coluna prism:doi e obtém os resumos
df_results['abstract'] = df_results['prism:doi'].apply(get_abstract)

# Configurando llm para analisar a base de dados

In [1]:
# Importando as bibliotecas necessário para llm com langchain
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'